In [9]:
#export
import torch
import torch.nn as nn
from fastai.vision.all import *
import time
from IPython.display import display, clear_output
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import regex as re
import numpy as np
from thesis.modules_th.video_block import * 
from thesis.modules_th.inflator import *
from thesis.modules_th.triplet_loss import *

In [ ]:
import torch
import torch.nn as nn
from fastai.vision.all import *
import time
from IPython.display import display, clear_output
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import regex as re
import numpy as np
from video_block import * 
from inflator import *
from triplet_loss import *

In [ ]:
#export
eugenio = True
path_data = '/mnt/data/eugeniomarinelli/videodata/df.csv'
df = pd.read_csv(path_data, index_col=0).dropna()

def get_vid_path(df:pd.Series): return df['paths']
def get_lbls(df:pd.Series): return df['lbl']

In [ ]:
df.head(1)

In [ ]:
#export
# Dataset creation
tfms = [[get_vid_path,Video.create,ResizeTime(l=20),Resize(224),ToTensor()],
        [get_lbls,Categorize()]]
splits = RandomSplitter(seed=42)(df)
dsets = Datasets(df, tfms, splits=splits)

In [ ]:
t0 = time.perf_counter()
bs = 16
for i, item in enumerate(dsets.train):
#     print(item)
    if i>=bs: break 
time.perf_counter() - t0


In [ ]:
#export
# Dataloaders
n_lbl, n_el = 4, 8
shuffle_fn = UniformizedShuffle(df['lbl'].iloc[splits[0]], n_lbl=n_lbl, n_el=n_el)
dls = dsets.dataloaders(bs=n_lbl*n_el, shuffle_fn=shuffle_fn, num_workers=10)

In [ ]:
#export
head = inflate(create_head(1024,157,lin_ftrs=[256]))
body = inflate(create_body(resnet34, cut=-2))
model = TLModel(body,head)
learn = Learner(dls, 
                model,
                loss_func=MixedLoss(alpha=1.0, margin=.2),
                splitter=my_splitter,
                metrics=tl_accuracy
               ).to_parallel()

In [ ]:
#export
with learn.distrib_ctx(): learn.fine_tune(8)
learn.save('trained_models/Inflated1')

In [ ]:
os.environ['WORLD_SIZE'] = '2'
os.environ['RANK'] = '0'
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '29500'

In [ ]:
from nbdev.export import *
notebook2script()

In [ ]:
# default_exp first_inflated_NN